In [29]:
import pandas as pd
import numpy as np
df=pd.read_csv('../Emission_computing/final_ingredients_emissions.csv')

In [30]:
df

,ingredient,emissions
0,allspice,1133
1,almond extract,656
2,almonds,2856
3,anchovies,4220
4,apple juice,2440
...,...,...
405,whiskey,2320
406,whiting,4362
407,wine,1369
408,worcestershire sauce,222


In [16]:
output_dict={'ingredient': ['chili', 'fresh ginger', 'garlic', 'cilantro', 'limes', 'salt', 'chile paste', 'red onion', 'sea bass', 'chili', 'cilantro', 'chili', 'sweet potato', 'fine sea salt'], 'value': [0.0, 0.06, 0.25, 1.0, 2.0, 0.12, 0.5, 0.25, 0.33, 0.0, 0.25, 0.25, 0.25, 0.0], 'metric': [None, 'piece', 'clove', 'sprig', '<unit>', 'teaspoon', 'teaspoon', '<unit>', 'pound', None, 'sprig', '<unit>', '<unit>', None]}

In [22]:
output_dict

{'ingredient': ['chili',
  'fresh ginger',
  'garlic',
  'cilantro',
  'limes',
  'salt',
  'chile paste',
  'red onion',
  'sea bass',
  'chili',
  'cilantro',
  'chili',
  'sweet potato',
  'fine sea salt'],
 'value': [0.0,
  0.06,
  0.25,
  1.0,
  2.0,
  0.12,
  0.5,
  0.25,
  0.33,
  0.0,
  0.25,
  0.25,
  0.25,
  0.0],
 'metric': [None,
  'piece',
  'clove',
  'sprig',
  '<unit>',
  'teaspoon',
  'teaspoon',
  '<unit>',
  'pound',
  None,
  'sprig',
  '<unit>',
  '<unit>',
  None]}

In [23]:
missing_values = ["<unit>", 'None']

In [18]:
def fill_empties(output_dict):
    output_df=pd.DataFrame(output_dict)
    output_df.metric=output_df.metric.astype(str)
    for item in missing_values:
        output_df.metric.replace(item, "g", inplace=True)
    return output_df

In [101]:
def match_ingredients(output_df):
    final_dict={'ingredient':[], 'emission':[]}
    missing_ingredients={'missing_ingredients':[]}
    



    
    for ingredient in output_df['ingredient']:
        # 1. Lemmatize 
        ingredient_words=re.split('\s+', ingredient)
        lemmatizer = WordNetLemmatizer()
        lemmatized_output = ' '.join([lemmatizer.lemmatize(w) for w in ingredient_words])
        
        # 2. Try the whole lemmatized sentence
        if df[df['ingredient'].str.match(r'.*'+str(lemmatized_output)+'.*')== True].ingredient.values.size>0:
                final_dict['ingredient'].append(df[df['ingredient'].str.match(r'.*'+str(lemmatized_output)+'.*')== True].ingredient.values[0])
                final_dict['emission'].append(df[df['ingredient'].str.match(r'.*'+str(lemmatized_output)+'.*')== True].emissions.values[0])
        
        # 3. If the lemmatized output does not work, try the words
        else:
            for word in ingredient_words:
                try:
                    final_dict['ingredient'].append(df[df['ingredient'].str.match(r'.*'+str(word)+'.*')== True].ingredient.values[0])
                    final_dict['emission'].append(df[df['ingredient'].str.match(r'.*'+str(word)+'.*')== True].emissions.values[0])
                except IndexError:
                    missing_ingredients['missing_ingredients'].append(ingredient)
            
    final_df=pd.DataFrame(final_dict)
    
    
    # Adding the columns "value" and metric from output_dict
    final_df=final_df.merge(output_df,on="ingredient")
#     final_df['value']=output_df['value']
#     final_df['metric']=output_df['metric']
    return final_df,missing_ingredients

In [94]:
def convert(final_df):
    output_df=final_df.merge(conv_df, on='metric')
    output_df["calculated gCO2e"]=output_df["emission"]*output_df["value"].astype('int64')*output_df["to_Kg_multiplier"]    
    return output_df

In [95]:
output_df=fill_empties(output_dict)

In [103]:
final_df,missing_ingredients=match_ingredients(output_df)

In [104]:
final_df

,ingredient,emission,value,metric
0,garlic,452,0.25,clove
1,salt,1133,0.12,teaspoon
2,salt,1133,0.12,teaspoon
3,red onion,1051,0.25,g
4,sea bass,4362,0.33,pound
5,sea bass,4362,0.33,pound
6,sweet potato,1273,0.25,g


In [42]:
output_df

,ingredient,value,metric
0,chili,0.00,g
1,fresh ginger,0.06,piece
2,garlic,0.25,clove
3,cilantro,1.00,sprig
4,limes,2.00,g
5,salt,0.12,teaspoon
6,chile paste,0.50,teaspoon
7,red onion,0.25,g
8,sea bass,0.33,pound
9,chili,0.00,g


In [64]:
df[df['ingredient'].str.match(r'^'+str("sweet potato")+'.*')== True].ingredient.values[0]

'sweet potato'

In [45]:
import re

In [46]:
re.split('\s+', "cherry tomatoes")

['cherry', 'tomatoes']

In [66]:
import nltk
from nltk.stem import WordNetLemmatizer 

In [70]:
lemmatizer = WordNetLemmatizer()

In [75]:
print(lemmatizer.lemmatize("sweet potatoes"))

sweet potatoes


In [72]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/mateo/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [84]:
if df[df['ingredient'].str.match(r'^'+str("word")+'.*')== True].ingredient.values.size>0:
    print('OK')
else:
    print('Not OK')

Not OK


# Solving the API issue

In [32]:
def load_image(img):
    im = Image.open(img)
    image = im.resize((224,224))    

    return image

In [33]:
img=load_image("../2014_Bruschetta_The_Larder_Chiang_Mai.jpg")

In [37]:
img=np.array(img)

In [38]:
img.shape

(224, 224, 3)

In [27]:
import PIL
from PIL import Image
import numpy as np
import json

In [13]:
X = img.reshape(img.shape[0]*img.shape[1]*img.shape[2])
X=X.tolist()
X_json=json.dumps(X)

In [22]:
import pickle

In [24]:
filehandler = open(b"../Image.obj","wb")
# pickle.dump(response_reshape,filehandler)

In [25]:
filehandler

<_io.BufferedWriter name=b'../Image.obj'>